In [1]:
from collections import Counter
from itertools import combinations
from math import sqrt
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" #comment out this line if you want to use gpu
import random
from keras.layers import Concatenate, Dense, Dot, Dropout, Embedding, Input, Reshape
from keras.models import Model
from keras.callbacks import Callback, ModelCheckpoint
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import tensorflow

Using TensorFlow backend.


### Set random seed

In [2]:
random.seed(2020)
np.random.seed(2020)
tensorflow.random.set_seed(2020)

### Root Mean Squared Error (RMSE) is used to evaluate the performance of a recommendation algorithm, so we need to define the following utility function to compute the RMSE given the predicted ratings and the ground truth ratings. 

In [3]:
'''
params:
    -pred: an array containing all predicted ratings
    -actual: an array containing all ground truth ratings
    
return:
    a scalar whose value is the rmse
'''
def rmse(pred, actual):
    # Ignore ratings with value zero.
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, actual))

# Implement Wide and Deep Learning (WDL) Model

### The wide component is a generalized linear model that takes in the raw input features and the cross-product transformation of categorical features, which enables it to learn the frequent co-occurrence of items or features. 

### The deep component is a Feed-forward Neural Network (FNN) which takes in both continuous and categorical features as input. Specifically,  the normalized values of continuous features are concatenated with the low-dimensional dense embedding vectors converted from categorical features. This concatenated vector is then fed into the FNN during each foward pass. This mechanism tend to increase the diversity of recommendations.

In [4]:
'''
params:
    -len_continuous: number of continuous features
    -deep_vocab_lens: an array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep 
        categorical feature
    -len_wide: number of wide features
    -embed_size: dimension of the embedding vectors of deep categorical features
    
return:
    a keras Model object for the constructed wdl model 
'''


def build_wdl_model(len_continuous, deep_vocab_lens, len_wide, embed_size):
    # A list containing all input layers
    input_list = []
    
    # Input layer for continuous features
    continuous_input = Input(shape=(len_continuous,), dtype='float32', name='continuous_input')
    input_list.append(continuous_input)
    
    
    # Get embeddings for all deep categorical features
    emb_list = []
    for vocab_size in deep_vocab_lens:
        _input = Input(shape=(1,), dtype='int32')
        input_list.append(_input)
        _emb = Embedding(output_dim=embed_size, input_dim=vocab_size, input_length=1)(_input)
        _emb = Reshape((embed_size,))(_emb)
        emb_list.append(_emb)
    

    # Create input layer for deep component by concatenating the embeddings and continuous features' input layer
    deep_input = Concatenate()(emb_list + [continuous_input])
    
    

    # Construct deep component
#     dense_0 = Dense(512, activation='relu')(deep_input)
#     dense_0_dp = Dropout(0.00)(dense_0)
#     dense_1 = Dense(128, activation='relu')(deep_input)
#     dense_1_dp = Dropout(0.00)(dense_1)
    dense_2 = Dense(128, activation='relu')(deep_input)
    dense_2_dp = Dropout(0.00)(dense_2)
    dense_3 = Dense(64, activation='relu')(dense_2_dp)
    dense_3_dp = Dropout(0.00)(dense_3)

    
    # Create input layer for wide component
    wide_input = Input(shape=(len_wide,), dtype='float32')
    input_list.append(wide_input)

    
    # Concatenate the outputs of deep and wide components and feed the 
    # concatenated vector into the finall fully connected layer
    fc_input = Concatenate()([dense_3_dp, wide_input])
    model_output = Dense(1)(fc_input)
    
    model = Model(inputs=input_list,
                  outputs=model_output)
    return model
    

# Utility functions to get the values of different types of features

### Continuous features

In [5]:
'''
params:
    -df: input dataframe
    -continuous_columns: column names of continuous features
    
return: 
    a numpy array where each row contains the values of continuous features in the corresponding row of the
    input dataframe
'''
def get_continuous_features(df, continuous_columns):
    continuous_features = df[continuous_columns].values
    return continuous_features

### Cross product transformation of categorical features

In [6]:
'''
params:
    -df: input dataframe
    -comb_p: number of elements in each combination (e.g., there are two elements in the combination {fried chicken, chicken and 
    waffle}, and three elements in the combination {fried chicken, chicken and waffle, chicken fried rice})
    -topk: number of mostly frequent combinations to retrieve
    -output_freq: whether to return the frequencies of retrieved combinations
    
return:
    1. output_freq = True: a list X where each element is a tuple containing a combinantion tuple and corresponding frequency, and the 
        elements are stored in the descending order of their frequencies
    2. output_freq = False: a list X where each element is a tuple containing a combinantion tuple, and the elements are stored in 
    the descending order of their frequencies
'''
def get_top_k_p_combinations(df, comb_p, topk, output_freq=False):
    def get_category_combinations(categories_str, comb_p=2):
        categories = categories_str.split(', ')
        return list(combinations(categories, comb_p))
    all_categories_p_combos = df["item_categories"].apply(
        lambda x: get_category_combinations(x, comb_p)).values.tolist()
    all_categories_p_combos = [tuple(t) for item in all_categories_p_combos for t in item]
    tmp = dict(Counter(all_categories_p_combos))
    sorted_categories_combinations = list(sorted(tmp.items(), key=lambda x: x[1], reverse=True))
    if output_freq:
        return sorted_categories_combinations[:topk]
    else:
        return [t[0] for t in sorted_categories_combinations[:topk]]

### Wide features

In [7]:
'''
params:
    -df: input dataframe
    -selected_categories_to_idx: a dictionary mapping item categories to corrresponding integral indices
    -top_combinations: a list containing retrieved mostly frequent combinantions of item categories
    
return:
    a numpy array where each row contains the categorical features' binary encodings and cross product
    transformations for the corresponding row of the input dataframe
'''

def get_wide_features(df, selected_categories_to_idx, top_combinations):
    def categories_to_binary_output(categories):
        binary_output = [0 for _ in range(len(selected_categories_to_idx))]
        for category in categories.split(', '):
            if category in selected_categories_to_idx:
                binary_output[selected_categories_to_idx[category]] = 1
            else:
                binary_output[0] = 1
        return binary_output
    def categories_cross_transformation(categories):
        current_category_set = set(categories.split(', '))
        corss_transform_output = [0 for _ in range(len(top_combinations))]
        for k, comb_k in enumerate(top_combinations):
            if len(current_category_set & comb_k) == len(comb_k):
                corss_transform_output[k] = 1
            else:
                corss_transform_output[k] = 0
        return corss_transform_output

    category_binary_features = np.array(df.item_categories.apply(
        lambda x: categories_to_binary_output(x)).values.tolist())
    category_corss_transform_features = np.array(df.item_categories.apply(
        lambda x: categories_cross_transformation(x)).values.tolist())
    return np.concatenate((category_binary_features, category_corss_transform_features), axis=1)


# Rating Prediction

### Load train, validation and test rating tables

In [8]:
tr_df = pd.read_csv("data/train.csv")
val_df = pd.read_csv("data/valid.csv")
te_df = pd.read_csv("data/test.csv")

tr_ratings = tr_df.stars.values
val_ratings = val_df.stars.values

### Load content feautures tables of users and items

In [9]:
user_df = pd.read_csv("data/user.csv")
item_df = pd.read_csv("data/business.csv")

# Rename some columns of dfs and convert the indices of dfs into string type for easier reference in later stage 
user_df = user_df.rename(index=str, columns={t: 'user_' + t for t in user_df.columns if t != 'user_id'})
item_df = item_df.rename(index=str, columns={t: 'item_' + t for t in item_df.columns if t != 'business_id'})

In [10]:
def get_num(x):
    if isinstance(x, str):
        return len(x.split(','))
    return 0

In [11]:
user_df['user_year'] = user_df.user_yelping_since.apply(lambda x: int(x.split('-')[0]) - 2000)
user_df['user_num_elite'] = user_df.user_elite.apply(lambda x: get_num(x))

In [12]:
def make_attr(name, d):
    if isinstance(d, str):
        d = eval(d)
        if name in d:
            return d[name]
    return 'unk'

In [13]:
item_df['item_RestaurantsAttire'] = item_df.item_attributes.apply(lambda x: make_attr('RestaurantsAttire', x))

item_df['item_WiFi'] = item_df.item_attributes.apply(lambda x: make_attr('WiFi', x))

item_df['item_BusinessAcceptsCreditCards'] = item_df.item_attributes.apply(lambda x: make_attr('BusinessAcceptsCreditCards', x))

item_df['item_Ambience'] = item_df.item_attributes.apply(lambda x: make_attr('Ambience', x))

item_df['item_NoiseLevel'] = item_df.item_attributes.apply(lambda x: make_attr('NoiseLevel', x))

item_df['item_GoodForKids'] = item_df.item_attributes.apply(lambda x: make_attr('GoodForKids', x))

item_df['item_RestaurantsPriceRange2'] = item_df.item_attributes.apply(lambda x: make_attr('RestaurantsPriceRange2', x))

item_df['item_GoodForMeal'] = item_df.item_attributes.apply(lambda x: make_attr('GoodForMeal', x))

item_df['item_BusinessParking'] = item_df.item_attributes.apply(lambda x: make_attr('BusinessParking', x))

item_df['item_RestaurantsGoodForGroups'] = item_df.item_attributes.apply(lambda x: make_attr('RestaurantsGoodForGroups', x))

item_df['item_Alcohol'] = item_df.item_attributes.apply(lambda x: make_attr('Alcohol', x))

item_df['item_RestaurantsReservations'] = item_df.item_attributes.apply(lambda x: make_attr('RestaurantsReservations', x))

### Associate each row in the rating tables with corresponding user's and item's content features through merging the rating tables and content features tables

In [17]:
# Save the original row indices of each rating table
tr_df["index"] = tr_df.index
val_df["index"]  = val_df.index
te_df["index"] = te_df.index

tr_df = pd.merge(pd.merge(tr_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
val_df = pd.merge(pd.merge(val_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)
te_df = pd.merge(pd.merge(te_df, user_df, on='user_id'), item_df, on='business_id').sort_values(by=['index']).reset_index(drop=True)


In [18]:
tr_df.columns

Index(['user_id', 'business_id', 'stars', 'index', 'user_Unnamed: 0',
       'user_name', 'user_review_count', 'user_yelping_since', 'user_useful',
       'user_funny', 'user_cool', 'user_elite', 'user_fans',
       'user_average_stars', 'user_compliment_hot', 'user_compliment_more',
       'user_compliment_profile', 'user_compliment_cute',
       'user_compliment_list', 'user_compliment_note', 'user_compliment_plain',
       'user_compliment_cool', 'user_compliment_funny',
       'user_compliment_writer', 'user_compliment_photos', 'user_year',
       'user_num_elite', 'item_Unnamed: 0', 'item_name', 'item_address',
       'item_city', 'item_state', 'item_postal_code', 'item_latitude',
       'item_longitude', 'item_stars', 'item_review_count', 'item_is_open',
       'item_attributes', 'item_categories', 'item_hours',
       'item_RestaurantsAttire', 'item_WiFi',
       'item_BusinessAcceptsCreditCards', 'item_Ambience', 'item_NoiseLevel',
       'item_GoodForKids', 'item_RestaurantsPr

In [19]:
user_df['user_fans'].value_counts()

0       572
1       503
2       337
3       212
4       163
       ... 
1242      1
1462      1
51        1
59        1
894       1
Name: user_fans, Length: 160, dtype: int64

### Prepare continuous features

In [20]:
# Specify the columns containing conitnuous features
# continuous_columns = ["user_average_stars", "user_cool", "user_fans", 
#                       "user_review_count", "user_useful", "user_funny",
#                       "item_is_open", "item_latitude", "item_longitude", 
#                       "item_review_count", "item_stars"]

continuous_columns = ["user_average_stars", "user_cool", "user_fans", 
                      "user_review_count", "user_useful", "user_funny",
                      "item_is_open", "item_latitude", "item_longitude", 
                      "item_review_count", "item_stars",
                      'user_compliment_hot', 'user_compliment_more',
                      'user_compliment_profile', 'user_compliment_cute',
                      'user_compliment_list', 'user_compliment_note', 'user_compliment_plain',
                      'user_compliment_cool', 'user_compliment_funny',
                      'user_compliment_writer', 'user_compliment_photos',
                      'user_year'# , 'user_num_elite'
                     ]

# Get values of continous features for train/validation/test sets using the utility function defined previously

tr_continuous_features = get_continuous_features(tr_df, continuous_columns)
val_continuous_features = get_continuous_features(val_df, continuous_columns)
te_continuous_features = get_continuous_features(te_df, continuous_columns)

# Standardize each feature by removing the mean of the training samples and scaling to unit variance.
# See https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html for more details.
scaler = StandardScaler().fit(tr_continuous_features)

tr_continuous_features = scaler.transform(tr_continuous_features)
val_continuous_features = scaler.transform(val_continuous_features)
te_continuous_features = scaler.transform(te_continuous_features)

In [21]:
tr_continuous_features.shape

(60080, 23)

### Prepare deep categorical features

In [22]:
# Sepcify column names of deep categorical features
# item_deep_columns = ["item_city", "item_postal_code", "item_state"]
item_deep_columns = ["item_city", "item_postal_code", "item_state", # original feaeture
                     'business_id', # 'item_name',
                     'item_RestaurantsAttire', 'item_WiFi',
                     'item_Ambience', 'item_NoiseLevel', # 'item_GoodForKids', 
                     'item_BusinessAcceptsCreditCards', 
                     'item_RestaurantsPriceRange2', 'item_GoodForMeal', 'item_BusinessParking', 'item_RestaurantsGoodForGroups']

# An array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep categorical feature
item_deep_vocab_lens = []

for col_name in item_deep_columns:
    # Get all unique values of this deep categorical feature
    tmp = item_df[col_name].unique()
    
    # Create a dictionary mapping each unique value to a unique integral index
    vocab = dict(zip(tmp, range(1, len(tmp) + 1)))
    
    # Get the number of unique values of this deep categorical features
    item_deep_vocab_lens.append(len(vocab) + 1)
    
    # Create a new column where each entry stores the integral index of this deep categorical feature's value in the same row
    item_df[col_name + "_idx"] = item_df[col_name].apply(lambda x: vocab[x])


# Create a dictionary mapping each business id to corresponding values of deep categorical features
item_deep_idx_columns = [t + "_idx" for t in item_deep_columns]
item_to_deep_categorical_features = dict(zip(item_df.business_id.values, item_df[item_deep_idx_columns].values.tolist()))

# Creat numpy arrays storing corresponding deep categorical features' values of train/validation/test sets using the above mapping
# tr_deep_categorical_features = np.array(tr_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
# val_deep_categorical_features = np.array(val_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())
# te_deep_categorical_features = np.array(te_df.business_id.apply(lambda x: item_to_deep_categorical_features[x]).values.tolist())

In [23]:
# Sepcify column names of deep categorical features
# item_deep_columns = ["item_city", "item_postal_code", "item_state"]
user_deep_columns = ["user_id", "user_elite"]

# An array of integers where deep_vocab_lens[i] represents the number of unique values of (i+1)-th deep categorical feature
user_deep_vocab_lens = []

for col_name in user_deep_columns:
    # Get all unique values of this deep categorical feature
    tmp = user_df[col_name].unique()
    
    # Create a dictionary mapping each unique value to a unique integral index
    vocab = dict(zip(tmp, range(1, len(tmp) + 1)))
    
    # Get the number of unique values of this deep categorical features
    user_deep_vocab_lens.append(len(vocab) + 1)
    
    # Create a new column where each entry stores the integral index of this deep categorical feature's value in the same row
    user_df[col_name + "_idx"] = user_df[col_name].apply(lambda x: vocab[x])


# Create a dictionary mapping each business id to corresponding values of deep categorical features
user_deep_idx_columns = [t + "_idx" for t in user_deep_columns]
user_to_deep_categorical_features = dict(zip(user_df.user_id.values, user_df[user_deep_idx_columns].values.tolist()))

tr_deep_categorical_features = np.array(tr_df.apply(lambda row: item_to_deep_categorical_features[row.business_id] + user_to_deep_categorical_features[row.user_id], axis=1).values.tolist())
val_deep_categorical_features = np.array(val_df.apply(lambda row: item_to_deep_categorical_features[row.business_id] + user_to_deep_categorical_features[row.user_id], axis=1).values.tolist())
te_deep_categorical_features = np.array(te_df.apply(lambda row: item_to_deep_categorical_features[row.business_id] + user_to_deep_categorical_features[row.user_id], axis=1).values.tolist())

In [24]:
tr_deep_categorical_features.shape

(60080, 15)

### Prepare wide features

##### Prepare binary encoding for each selected category

In [34]:
# Collect the categories of all items 
all_categories = [category for category_list in item_df.item_categories.values for category in category_list.split(", ")]

# Sort all unique values of the item categories by their frequencies in descending order
category_sorted = sorted(Counter(all_categories).items(), key=lambda x: x[1], reverse=True)

# Select top 500 most frequent categories
selected_categories = [t[0] for t in category_sorted[:]]

# Create a dictionary mapping each secleted category to a unique integral index
selected_categories_to_idx = dict(zip(selected_categories, range(1, len(selected_categories) + 1)))

# Map all categories unseen in the item df to index 0
selected_categories_to_idx['unk'] = 0

# Create a dictionary mapping each integral index to corresponding category
idx_to_selected_categories = {val: key for key, val in selected_categories_to_idx.items()}


##### Prepare cross product transformation for categories

In [35]:
# Get most frequent categories combinantions using the utility function defined previously and store them in the folloing list
top_combinations = []

# Get top 50 most frequent two-categories combinantions in the train set

top_combinations += get_top_k_p_combinations(tr_df, 2, 100, output_freq=False)

# Get top 30 most frequent three-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 3, 60, output_freq=False)

# Get top 20 most frequent four-categories combinantions in the train set
top_combinations += get_top_k_p_combinations(tr_df, 4, 40, output_freq=False)

# # Get top 10 most frequent five-categories combinantions in the train set
# top_combinations += get_top_k_p_combinations(tr_df, 5, 20, output_freq=False)

# Convert each combinantion in the list to a set data structure
top_combinations = [set(t) for t in top_combinations]

In [36]:
# Get values of wide features for train/validation/test sets using the utility function defined previously

tr_wide_features = get_wide_features(tr_df, selected_categories_to_idx, top_combinations)
val_wide_features = get_wide_features(val_df, selected_categories_to_idx, top_combinations)
te_wide_features = get_wide_features(te_df, selected_categories_to_idx, top_combinations)


In [37]:
tr_wide_features.shape

(60080, 806)

### Build the input list for each of the train/validation/test sets through aggregating all continuous, deep categorical and wide features


In [38]:
tr_features = []
tr_features.append(tr_continuous_features.tolist())
tr_features += [tr_deep_categorical_features[:,i].tolist() for i in range(tr_deep_categorical_features.shape[1])]
tr_features.append(tr_wide_features.tolist())


val_features = []
val_features.append(val_continuous_features.tolist())
val_features += [val_deep_categorical_features[:,i].tolist() for i in range(val_deep_categorical_features.shape[1])]
val_features.append(val_wide_features.tolist())

te_features = []
te_features.append(te_continuous_features.tolist())
te_features += [te_deep_categorical_features[:,i].tolist() for i in range(te_deep_categorical_features.shape[1])]
te_features.append(te_wide_features.tolist())

In [39]:
len(tr_features)

17

### Build the WDL model defined above

In [114]:
wdl_model = build_wdl_model(
        len(tr_continuous_features[0]),
        item_deep_vocab_lens + user_deep_vocab_lens,  
        len(tr_wide_features[0]), 
        embed_size=50
)

### Train the model using Adagrad optimizer and mean squared error loss

In [115]:
# random.seed(2020)
# np.random.seed(2020)
# tensorflow.random.set_seed(2020)

wdl_model.compile(optimizer='adagrad', loss='mse')

history = wdl_model.fit(
        tr_features, 
        tr_ratings, 
        epochs=1,
        verbose=1,
        callbacks=[ModelCheckpoint('model.h5')])

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
60080/60080 [==============================] - 17s 280us/step - loss: 1.0605


### Evaluate the model on train and validation sets using RMSE¶

In [47]:
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  1.0507568554059001
VALID RMSE:  1.0745017814946722


In [67]:
# add all continious features to wide features
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  1.032725657788163
VALID RMSE:  1.0563793314017844


In [107]:
# add item name to deep features & feature top 600
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9930970856437658
VALID RMSE:  1.0423679484274364


In [138]:
# feature top 500; mbinations: 100, 60, 40
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9883511022944678
VALID RMSE:  1.0380338781159348


In [287]:
# feature top 600; mbinations: 100, 60, 40
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9900975883817927
VALID RMSE:  1.037506221509891


In [303]:
# fix random seed 2020
# feature top 600; mbinations: 100, 60, 40
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.998233966865175
VALID RMSE:  1.0458363976195457


In [334]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; mbinations: 100, 60, 40
# Dropout = 0.2
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9914249484465553
VALID RMSE:  1.0380548550938558


In [345]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; mbinations: 100, 60, 40
# Dropout = 0.1
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9914174485947024
VALID RMSE:  1.0359051369295789


In [353]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; combinations: 100, 60, 40
# Dropout = 0
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9852044000928517
VALID RMSE:  1.0311300320996717


In [32]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; combinations: 100, 60, 40
# Dropout = 0
# embedding = 64
# add item cactegorical features: ......
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9884606660367468
VALID RMSE:  1.0300729685332184


In [154]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; combinations: 100, 60, 40
# Dropout = 0
# embedding = 32
# add item cactegorical features: ......A lot
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  1.0010040428362543
VALID RMSE:  1.0293079402562368


In [29]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; combinations: 100, 60, 40
# Dropout = 0
# embedding = 32
# add item cactegorical features: ......A lot
# add user_id + business_id info
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9825471506672215
VALID RMSE:  1.02930971737958


In [32]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; combinations: 100, 60, 40
# Dropout = 0
# embedding = 64
# add item cactegorical features: ......A lot
# add user_id + business_id info
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9819977387550399
VALID RMSE:  1.0282094441116103


In [38]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 600; combinations: 100, 60, 40
# Dropout = 0
# embedding = 50
# add item cactegorical features: ......A lot
# add user_id + business_id info
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.980885742862147
VALID RMSE:  1.02732847268181


In [42]:
# all current continious & categorical features
# fix random seed 2020
# cate feature top 606; combinations: 100, 60, 40
# Dropout = 0
# embedding = 50
# add item cactegorical features: ......A lot
# add user_id + business_id info
# deep components layers: 128, 64
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9809739433969699
VALID RMSE:  1.0271979860205405


#### Formal output with unset random seed - 5 times

In [45]:
# 1
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9728837378504239
VALID RMSE:  1.0226843085225688


In [74]:
y_pred_val = wdl_model.predict(val_features)
val_df_pred = val_df[['user_id', 'business_id']]
val_df_pred['stars'] = y_pred_val
val_df_pred.to_csv('val_pre_1.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [72]:
y_pred_te = wdl_model.predict(te_features)
te_df_pred = te_df[['user_id', 'business_id']]
te_df_pred['stars'] = y_pred_te
te_df_pred.to_csv('te_pre_1.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [79]:
# 2
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.977544920488189
VALID RMSE:  1.0252415830075328


In [80]:
y_pred_val = wdl_model.predict(val_features)
val_df_pred = val_df[['user_id', 'business_id']]
val_df_pred['stars'] = y_pred_val
val_df_pred.to_csv('val_pre_2.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [81]:
y_pred_te = wdl_model.predict(te_features)
te_df_pred = te_df[['user_id', 'business_id']]
te_df_pred['stars'] = y_pred_te
te_df_pred.to_csv('te_pre_2.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [90]:
# 3
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.976045028585581
VALID RMSE:  1.0266512934518501


In [91]:
y_pred_val = wdl_model.predict(val_features)
val_df_pred = val_df[['user_id', 'business_id']]
val_df_pred['stars'] = y_pred_val
val_df_pred.to_csv('val_pre_3.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [92]:
y_pred_te = wdl_model.predict(te_features)
te_df_pred = te_df[['user_id', 'business_id']]
te_df_pred['stars'] = y_pred_te
te_df_pred.to_csv('te_pre_3.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [111]:
# 4
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9787179092755424
VALID RMSE:  1.0245971891429382


In [112]:
y_pred_val = wdl_model.predict(val_features)
val_df_pred = val_df[['user_id', 'business_id']]
val_df_pred['stars'] = y_pred_val
val_df_pred.to_csv('val_pre_4.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [113]:
y_pred_te = wdl_model.predict(te_features)
te_df_pred = te_df[['user_id', 'business_id']]
te_df_pred['stars'] = y_pred_te
te_df_pred.to_csv('te_pre_4.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [116]:
# 5
y_pred = wdl_model.predict(tr_features)
print("TRAIN RMSE: ", rmse(y_pred, tr_ratings))
y_pred = wdl_model.predict(val_features)
print("VALID RMSE: ", rmse(y_pred, val_ratings))

TRAIN RMSE:  0.9763686275005208
VALID RMSE:  1.0253317879018073


In [117]:
y_pred_val = wdl_model.predict(val_features)
val_df_pred = val_df[['user_id', 'business_id']]
val_df_pred['stars'] = y_pred_val
val_df_pred.to_csv('val_pre_5.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [118]:
y_pred_te = wdl_model.predict(te_features)
te_df_pred = te_df[['user_id', 'business_id']]
te_df_pred['stars'] = y_pred_te
te_df_pred.to_csv('te_pre_5.csv', index=False)

C:\Users\Kang\Anaconda3\envs\comp4332\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
